In [9]:
''' Defines the Weather class. '''

import time
import json

from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError
from pyowm.exceptions.api_call_error import APIInvalidSSLCertificateError

from config import OWM_API_key_loohoo as loohoo_key
from config import OWM_API_key_masta as masta_key


class Weather:
    ''' A dictionary of weather variables and their observed/forecasted values
    for a given instant in time at a specified location.
    '''
    
    def __init__(self, location, _type, data=None):
        '''
        :param location: can be either valid US zipcode or coordinate dictionary
        :type location: If this param is a zipcode, it should be str, otherwise
        dict
        :param _type: Indicates whether its data is observational or forecasted
        :type _type: string  It must be either 'observation' or 'forecast'
        '''

        self.type = _type
        self.loc = location
        # I think you don't need this for anything
#         self.creation_time = int(time.time() // 1)
        self.weather = data

        # make the _id for each weather according to its reference time
        if _type == 'forecast' and 'reference_time' in data:
            self._id = f'{str(location)}{str(data["reference_time"])}'
        elif _type == 'observation' and 'reference_time' in data:
            self._id = f'{str(location)}{str(10800 * (data["reference_time"]//10800 + 1))}'
        
        self.as_dict = {'_id': self._id,
                       '_type': self.type,
                        'weather': self.weather
                       }
        
    def to_inst(self):
        ''' This will find the id'd Instant and add the Weather to it according 
        to its type. '''
        
        from instant import Instant
        
#         weather = self.as_dict
        if self.type == 'observation':
            _id = self._id
            instants.setdefault(_id, Instant(_id, observations=self.weather))
#             instants[_id]['observation'] = weather
            return
        if self.type == 'forecast':
            _id = self._id
            instants[_id]['forecasts'].append(weather)
            return


# def get_data_from_weather_api(owm, zipcode=None, coords=None):
def get_data_from_weather_api(owm, location):
    ''' Makes api calls for observations and forecasts and handles the API call errors.

    :param owm: the OWM API object
    :type owm: pyowm.OWM
    :param zipcode: the zipcode reference for the API call
    :type zipcode: string
    :param coords: the latitude and longitude coordinates reference for the API call
    :type coords: dict or 2-tuple that has 'lat' and 'lon' in it.

    returns: the API data
    '''
    
    result = None
    tries = 1
    while result is None and tries < 4:
        try:
            if type(location) == dict:
                result = owm.three_hours_forecast_at_coords(**location)
                return result
            elif type(location) == str:
                print('get data from weather api says location is a string')
                result = owm.weather_at_zip_code(location, 'us')
                return result
        except APIInvalidSSLCertificateError:
            loc = zipcode or 'lat: {}, lon: {}'.format(coords['lat'], coords['lon'])
            print(f'SSL error with {loc} on attempt {tries} ...trying again')
            if type(location) == dict:
                owm_loohoo = OWM(loohoo_key)
                owm = owm_loohoo
            elif type(location) == str:
                owm_masta = OWM(masta_key)
                owm = owm_masta
        except APICallTimeoutError:
            loc = location[:] or 'lat: {}, lon: {}'.format(location['lat'], location['lon'])
            print(f'Timeout error with {loc} on attempt {tries}... waiting 1 second then trying again')
            time.sleep(1)
        tries += 1
    if tries == 4:
        print('tried 3 times without response; breaking out and causing an error that will crash your current colleciton process...fix that!')
        return ### sometime write something to keep track of the zip and instant that isn't collected ###

def get_current_weather(location):
    ''' Get the current weather for the given zipcode or coordinates.

    :param code: the zip code to find weather data about
    :type code: string
    :param coords: the coordinates for the data you want
    :type coords: 2-tuple

    :return: the raw weather object
    :type: json
    '''
    owm = OWM(loohoo_key)

    m = 0
    # Try several times to get complete the API request
    while m < 4:
        try:
            # get the raw data from the OWM and make a Weather from its weather
            result = get_data_from_weather_api(owm, location)
            result = json.loads(result.to_JSON()) # the current weather for the given zipcode
            weather = Weather(location, 'observation', result['Weather'])
            return weather
        except APICallTimeoutError:
            owm = owm_loohoo
            m += 1
    print(f'Did not get current weather for {location} and reset owm')
    return ### after making the weather class, return one of them ###
    
def five_day(location):
    ''' Get each weather forecast for the corrosponding coordinates
    
    :param coords: the latitude and longitude for which that that weather is being forecasted
    :type coords: tuple containing the latitude and logitude for the forecast

    :return casts: the five day, every three hours, forecast for the zip code
    :type casts: dict
    '''

    owm = OWM(masta_key)

    Forecast = get_data_from_weather_api(owm, location).get_forecast()
    forecast = json.loads(Forecast.to_JSON())
    casts = [] # This is for the weather objects created in the for loop below.
    for data in forecast['weathers']:
        # Make an _id for the next Weather to be created, create the weather, 
        # and append it to the casts list.
        instant = data['reference_time']
        data['_id'] = f'{str(location)}{str(instant)}'
        casts.append(Weather(location, 'forecast', data))
    return casts


In [10]:
OWM_API_key_masta = 'ec7a9ff0f4a568d9e8e6ef8b810c599e'
OWM_API_key_loohoo ='ccf670fd173f90d5ae9c84ef6372573d'

weather = get_current_weather('27606')

get data from weather api says location is a string


In [11]:
five_day({'lon': -78.71, 'lat': 35.76})

In [12]:
{'reference_time': 1588843902,
  'sunset_time': 1588896845,
  'sunrise_time': 1588846952,
  'clouds': 0,
  'rain': {},
  'snow': {},
  'wind': {'speed': 2.26, 'deg': 318},
  'humidity': 89,
  'pressure': {'press': 1013, 'sea_level': None},
  'temperature': {'temp': 274.88,
   'temp_kf': None,
   'temp_max': 275.93,
   'temp_min': 274.26},
  'status': 'Clear',
  'detailed_status': 'clear sky',
  'weather_code': 800,
  'weather_icon_name': '01n',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None}
{'reference_time': 1589274000,
  'sunset_time': 0,
  'sunrise_time': 0,
  'clouds': 70,
  'rain': {},
  'snow': {},
  'wind': {'speed': 3.34, 'deg': 347},
  'humidity': 60,
  'pressure': {'press': 1023, 'sea_level': 1023},
  'temperature': {'temp': 280.14,
   'temp_kf': 0,
   'temp_max': 280.14,
   'temp_min': 280.14},
  'status': 'Clouds',
  'detailed_status': 'broken clouds',
  'weather_code': 803,
  'weather_icon_name': '04n',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None}
weather._id

'276061588863600'

In [13]:
weather.as_dict

{'_id': '276061588863600',
 '_type': 'observation',
 'weather': {'reference_time': 1588859625,
  'sunset_time': 1588896398,
  'sunrise_time': 1588846564,
  'clouds': 0,
  'rain': {},
  'snow': {},
  'wind': {'speed': 0.89, 'deg': 324, 'gust': 2.68},
  'humidity': 49,
  'pressure': {'press': 1016, 'sea_level': None},
  'temperature': {'temp': 286.06,
   'temp_kf': None,
   'temp_max': 287.59,
   'temp_min': 284.26},
  'status': 'Clear',
  'detailed_status': 'clear sky',
  'weather_code': 800,
  'weather_icon_name': '01d',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None}}

In [14]:
from Extract.make_instants import client, find_data
# set database and collection for testing
database = 'test'
collection = 'instant_temp'
# create a dict to hold the instants pulled from the database
instants = {}
data = find_data(client, database, collection)
# add each doc to instants and set its key and _id to the same values
for item in data:
    instants[f'{item["_id"]}'] = item['_id']
print(len(instants))

78706


In [15]:
weather.to_inst()

In [8]:
len(instants)

78707

In [9]:
weather = get_current_weather('27006')

get data from weather api says location is a string


In [10]:
weather.to_inst()

In [11]:
instants[weather._id].as_dict

{'_id': '270061588843902',
 'forecasts': list,
 'observations': {'reference_time': 1588843902,
  'sunset_time': 1588896845,
  'sunrise_time': 1588846952,
  'clouds': 0,
  'rain': {},
  'snow': {},
  'wind': {'speed': 2.26, 'deg': 318},
  'humidity': 89,
  'pressure': {'press': 1013, 'sea_level': None},
  'temperature': {'temp': 274.88,
   'temp_kf': None,
   'temp_max': 275.93,
   'temp_min': 274.26},
  'status': 'Clear',
  'detailed_status': 'clear sky',
  'weather_code': 800,
  'weather_icon_name': '01n',
  'visibility_distance': None,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None}}